In [1]:
import os
import random as rand
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from numpy import asarray
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score
from sklearn.datasets import fetch_lfw_people
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, RMSprop, Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding, BatchNormalization, Conv2D, MaxPool2D, Activation, MaxPooling2D, Dropout,AveragePooling2D, Flatten
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array


2022-12-08 11:42:27.926408: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
model = tf.keras.models.load_model('cnn3.h5')

2022-12-08 11:42:30.971675: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
train_dataset = tf.keras.utils.image_dataset_from_directory("/Users/cedricbeugre/Desktop/Deep-learning-Project-1/Dataset/Train",batch_size=32,image_size=(256, 256),shuffle=True)
test_dataset = tf.keras.utils.image_dataset_from_directory("/Users/cedricbeugre/Desktop/Deep-learning-Project-1/Dataset/Test",batch_size=32,image_size=(256, 256),shuffle=True)

class_names = train_dataset.class_names
print(class_names)

Found 12396 files belonging to 3 classes.
Found 3373 files belonging to 3 classes.
['Incorrectly', 'Withmask', 'Withoutmask']


In [6]:
import numpy as np
import cv2

cap = cv2.VideoCapture(0)
modelFile = "res10_300x300_ssd_iter_140000.caffemodel"
configFile = "deploy.prototxt"

while(True):
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    net = cv2.dnn.readNetFromCaffe(configFile, modelFile)
    h, w = frame.shape[:2]
    
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 117.0, 123.0))
    net.setInput(blob)
    faces = net.forward()

    for i in range(0, faces.shape[2]):
        confidence = faces[0, 0, i, 2]
        if confidence > 0.5:
            box = faces[0, 0, i, 3:7] * np.array([w, h, w, h])
            
            (x, y, x1, y1) = box.astype("int")
            
            
            face = frame[y:y1, x:x1]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (256, 256))
            face = img_to_array(face)
            face = preprocess_input(face)
            face = np.expand_dims(face, axis=0)
            preds = model.predict(face)

            if preds[0].argmax() == 0:
                cv2.rectangle(frame, (x, y-40), (x1, y), (0, 255, 255), -1)
                cv2.rectangle(frame, (x, y), (x1, y1), (0, 255, 255), 3)
                cv2.putText(frame, 'Incorrectly Masked', (x+10, y-9), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,0), 2)
            elif preds[0].argmax() == 1:
                cv2.rectangle(frame, (x, y-40), (x1, y), (0, 255, 0), -1)
                cv2.rectangle(frame, (x, y), (x1, y1), (0, 255, 0), 3)
                cv2.putText(frame, 'Correctly Masked', (x+10, y-9), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,0), 2)
            elif preds[0].argmax() == 2:
                cv2.rectangle(frame, (x, y-40), (x1, y), (0, 0, 255), -1)
                cv2.rectangle(frame, (x, y), (x1, y1), (0, 0, 255), 3)
                cv2.putText(frame, 'No Mask', (x+10, y-9), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,0), 2)
            print(class_names[preds[0].argmax()])
            
            
 
    cv2.imshow('Webcam',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

   

 

1/1 [==============================] - 0s 59ms/step
Incorrectly
1/1 [==============================] - 0s 61ms/step
Incorrectly
1/1 [==============================] - 0s 60ms/step
Incorrectly
1/1 [==============================] - 0s 59ms/step
Incorrectly
1/1 [==============================] - 0s 69ms/step
Incorrectly
1/1 [==============================] - 0s 70ms/step
Incorrectly
1/1 [==============================] - 0s 59ms/step
Incorrectly
1/1 [==============================] - 0s 58ms/step
Incorrectly
1/1 [==============================] - 0s 62ms/step
Incorrectly
1/1 [==============================] - 0s 61ms/step
Incorrectly
1/1 [==============================] - 0s 60ms/step
Incorrectly
1/1 [==============================] - 0s 57ms/step
Incorrectly
1/1 [==============================] - 0s 59ms/step
Incorrectly
1/1 [==============================] - 0s 60ms/step
Incorrectly
1/1 [==============================] - 0s 64ms/step
Incorrectly
1/1 [==============================] - 0

: 